In [1]:
import pyodbc
import pandas as pd
import numpy as np
import configparser

import mlflow.keras
from mlflow import log_metric, log_param, log_artifacts

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import matplotlib.pyplot as plt

import xgboost as xgb

Using TensorFlow backend.


In [2]:
lag_list = [5, 50, 100, 200]
epoch_list = [50,100,200,500]

ml_flow_name = 'HdM_Master_BI'

In [3]:
config = configparser.ConfigParser()
config.sections()
config.read('C:/tmp/BI.ini')

['C:/tmp/BI.ini']

# ETL

## Extract

In [4]:
def extract():
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server='config['Credentials']['Server']';'
                          'Database='config['Credentials']['Database']';'
                          'uid='config['Credentials']['User']';pwd='config['Credentials']['Password'])

    cursor = conn.cursor()
    query = 'select * from dbo.F_Monthly_Sales as F inner join dbo.D_Product on dbo.D_Product.ID_Product =F.ID_Product where F.ID_Product = 70'
    sql_query = pd.read_sql_query(query,conn)
    return sql_query

## Transform

In [6]:
def gen_features(X):    
    X['mean'] = X['Sales_Amount'].mean()
    X['skew'] = X['Sales_Amount'].skew()
    X['kurtosis'] = X['Sales_Amount'].kurtosis()
    X['std'] = X['Sales_Amount'].std()
    X['min'] = X['Sales_Amount'].min()
    X['max'] = X['Sales_Amount'].max()
    X['q95'] = X['Sales_Amount'].quantile(0.95)
    X['q99'] = X['Sales_Amount'].quantile(0.99)
    X['q05'] = X['Sales_Amount'].quantile(0.05)
    X['q01'] = X['Sales_Amount'].quantile(0.01)    
    return X

In [7]:
def transform(df, lag):
    label = df['Sales_Amount']
    label = label[1:].reset_index()
    label = label['Sales_Amount']
    label = label[lag:]
    
    df = df[:-1].drop('Unnamed: 0', axis=1)
    df = pd.DataFrame(df['Sales_Amount'])
    for lag in range(1, number_lags + 1):
        df['lag_' + str(lag)] = df.Sales_Amount.shift(lag)
    df = df[lag:]
    
    df = gen_features(df)
    
    df['label'] = label
    return df

## Load

In [8]:
def load(df):
    label = df['label']
    return df.drop('label', axis=1).reset_index().drop(['index', 'Sales_Amount'], axis=1), label.reset_index().drop('index', axis=1)

## ETL

In [9]:
def etl(lag):
    df = extract()
    df = transform(df, lag)
    return load(df)

# Model

In [10]:
def fit_model(X_train, y_train, lag, epoch):
    model = Sequential()
    model.add(Dense(lag, input_shape=(lag,)))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.compile(loss="mse", optimizer="adam", metrics=["accuracy", "mse"])
    
    model.fit(X_train, y_train, batch_size=10, epochs=epoch, verbose=1, validation_split=0.1)    

In [11]:
for number_lags in lag_list:
    for epoch in epoch_list:
        # ETL
        X, y = etl(number_lags)
        # Train Test Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        # Train
        mlflow.set_experiment(experiment_name=ml_flow_name)
        with mlflow.start_run(run_name='Neural Network'):
            mlflow.keras.autolog() 
            mlflow.log_param("lag", number_lags)

            fit_model(X_train, y_train, number_lags+10, epoch)    

INFO: 'HdM_Master_BI' does not exist. Creating a new experiment
Train on 2076 samples, validate on 231 samples
Epoch 1/50
2076/2076 [==============================] - 0s 158us/step - loss: 101.2632 - accuracy: 0.0530 - mse: 101.2632 - val_loss: 54.3978 - val_accuracy: 0.0476 - val_mse: 54.3978
Epoch 2/50
2076/2076 [==============================] - 0s 112us/step - loss: 78.2481 - accuracy: 0.0684 - mse: 78.2481 - val_loss: 45.4615 - val_accuracy: 0.1082 - val_mse: 45.4615
Epoch 3/50
2076/2076 [==============================] - 0s 113us/step - loss: 75.5135 - accuracy: 0.0848 - mse: 75.5135 - val_loss: 44.0406 - val_accuracy: 0.0649 - val_mse: 44.0406
Epoch 4/50
2076/2076 [==============================] - 0s 97us/step - loss: 74.4893 - accuracy: 0.0814 - mse: 74.4893 - val_loss: 43.0099 - val_accuracy: 0.1688 - val_mse: 43.0099
Epoch 5/50
2076/2076 [==============================] - 0s 105us/step - loss: 74.3345 - accuracy: 0.0949 - mse: 74.3345 - val_loss: 42.3879 - val_accuracy: 0.06

2076/2076 [==============================] - 0s 117us/step - loss: 71.2651 - accuracy: 0.0930 - mse: 71.2651 - val_loss: 45.4345 - val_accuracy: 0.0303 - val_mse: 45.4345
Epoch 46/50
2076/2076 [==============================] - 0s 118us/step - loss: 70.9929 - accuracy: 0.0930 - mse: 70.9930 - val_loss: 42.2178 - val_accuracy: 0.1342 - val_mse: 42.2178
Epoch 47/50
2076/2076 [==============================] - 0s 149us/step - loss: 71.3848 - accuracy: 0.0901 - mse: 71.3848 - val_loss: 41.9112 - val_accuracy: 0.0996 - val_mse: 41.9112
Epoch 48/50
2076/2076 [==============================] - 0s 121us/step - loss: 71.5811 - accuracy: 0.0934 - mse: 71.5811 - val_loss: 43.1980 - val_accuracy: 0.0823 - val_mse: 43.1980
Epoch 49/50
2076/2076 [==============================] - 0s 123us/step - loss: 71.1024 - accuracy: 0.0915 - mse: 71.1024 - val_loss: 43.1859 - val_accuracy: 0.1082 - val_mse: 43.1859
Epoch 50/50
2076/2076 [==============================] - 0s 116us/step - loss: 70.8123 - accuracy

2076/2076 [==============================] - 0s 94us/step - loss: 73.7469 - accuracy: 0.0901 - mse: 73.7469 - val_loss: 45.3258 - val_accuracy: 0.0260 - val_mse: 45.3258
Epoch 40/100
2076/2076 [==============================] - 0s 92us/step - loss: 74.9588 - accuracy: 0.0910 - mse: 74.9588 - val_loss: 42.5917 - val_accuracy: 0.2251 - val_mse: 42.5917
Epoch 41/100
2076/2076 [==============================] - 0s 89us/step - loss: 75.4722 - accuracy: 0.0910 - mse: 75.4723 - val_loss: 44.1738 - val_accuracy: 0.1818 - val_mse: 44.1738
Epoch 42/100
2076/2076 [==============================] - 0s 92us/step - loss: 73.9545 - accuracy: 0.0882 - mse: 73.9545 - val_loss: 42.4233 - val_accuracy: 0.2251 - val_mse: 42.4233
Epoch 43/100
2076/2076 [==============================] - 0s 103us/step - loss: 74.0000 - accuracy: 0.0973 - mse: 74.0000 - val_loss: 42.2044 - val_accuracy: 0.2078 - val_mse: 42.2044
Epoch 44/100
2076/2076 [==============================] - 0s 102us/step - loss: 74.0409 - accurac

Epoch 84/100
2076/2076 [==============================] - 0s 95us/step - loss: 72.2033 - accuracy: 0.0978 - mse: 72.2033 - val_loss: 40.6075 - val_accuracy: 0.0736 - val_mse: 40.6075
Epoch 85/100
2076/2076 [==============================] - 0s 92us/step - loss: 72.8633 - accuracy: 0.0862 - mse: 72.8633 - val_loss: 41.1363 - val_accuracy: 0.0606 - val_mse: 41.1363
Epoch 86/100
2076/2076 [==============================] - 0s 90us/step - loss: 72.8081 - accuracy: 0.0857 - mse: 72.8081 - val_loss: 42.3804 - val_accuracy: 0.1948 - val_mse: 42.3804
Epoch 87/100
2076/2076 [==============================] - 0s 95us/step - loss: 72.4377 - accuracy: 0.1002 - mse: 72.4377 - val_loss: 42.9915 - val_accuracy: 0.0433 - val_mse: 42.9915
Epoch 88/100
2076/2076 [==============================] - 0s 95us/step - loss: 72.8582 - accuracy: 0.0857 - mse: 72.8583 - val_loss: 40.9216 - val_accuracy: 0.1515 - val_mse: 40.9216
Epoch 89/100
2076/2076 [==============================] - 0s 96us/step - loss: 73.789

Epoch 27/200
2076/2076 [==============================] - 0s 94us/step - loss: 249.1517 - accuracy: 0.0000e+00 - mse: 249.1517 - val_loss: 209.1749 - val_accuracy: 0.0000e+00 - val_mse: 209.1749
Epoch 28/200
2076/2076 [==============================] - 0s 91us/step - loss: 249.1517 - accuracy: 0.0000e+00 - mse: 249.1517 - val_loss: 209.1749 - val_accuracy: 0.0000e+00 - val_mse: 209.1749
Epoch 29/200
2076/2076 [==============================] - 0s 98us/step - loss: 249.1517 - accuracy: 0.0000e+00 - mse: 249.1517 - val_loss: 209.1749 - val_accuracy: 0.0000e+00 - val_mse: 209.1749
Epoch 30/200
2076/2076 [==============================] - 0s 93us/step - loss: 249.1517 - accuracy: 0.0000e+00 - mse: 249.1517 - val_loss: 209.1749 - val_accuracy: 0.0000e+00 - val_mse: 209.1749
Epoch 31/200
2076/2076 [==============================] - 0s 92us/step - loss: 249.1517 - accuracy: 0.0000e+00 - mse: 249.1517 - val_loss: 209.1749 - val_accuracy: 0.0000e+00 - val_mse: 209.1749
Epoch 32/200
2076/2076 [=

...

Epoch 302/500
1958/1958 [==============================] - 0s 200us/step - loss: 1.7138 - accuracy: 0.3958 - mse: 1.7138 - val_loss: 67.0632 - val_accuracy: 0.0734 - val_mse: 67.0632
Epoch 303/500
1958/1958 [==============================] - 0s 195us/step - loss: 2.2625 - accuracy: 0.3458 - mse: 2.2625 - val_loss: 61.3259 - val_accuracy: 0.0413 - val_mse: 61.3259
Epoch 304/500
1958/1958 [==============================] - 0s 188us/step - loss: 1.7096 - accuracy: 0.4244 - mse: 1.7096 - val_loss: 60.0923 - val_accuracy: 0.0596 - val_mse: 60.0923
Epoch 305/500
1958/1958 [==============================] - 0s 192us/step - loss: 1.2958 - accuracy: 0.5107 - mse: 1.2958 - val_loss: 67.1895 - val_accuracy: 0.0367 - val_mse: 67.1895
Epoch 306/500
1958/1958 [==============================] - 0s 190us/step - loss: 2.6464 - accuracy: 0.2988 - mse: 2.6464 - val_loss: 65.5263 - val_accuracy: 0.0550 - val_mse: 65.5263
Epoch 307/500
1958/1958 [==============================] - 0s 210us/step - loss: 1.61

Epoch 347/500
1958/1958 [==============================] - 0s 191us/step - loss: 1.6825 - accuracy: 0.3927 - mse: 1.6825 - val_loss: 65.8213 - val_accuracy: 0.0642 - val_mse: 65.8213
Epoch 348/500
1958/1958 [==============================] - 0s 158us/step - loss: 1.8090 - accuracy: 0.4045 - mse: 1.8090 - val_loss: 63.5711 - val_accuracy: 0.0505 - val_mse: 63.5711
Epoch 349/500
1958/1958 [==============================] - 0s 148us/step - loss: 0.9990 - accuracy: 0.5296 - mse: 0.9990 - val_loss: 66.9067 - val_accuracy: 0.0367 - val_mse: 66.9067
Epoch 350/500
1958/1958 [==============================] - 0s 153us/step - loss: 1.3328 - accuracy: 0.4408 - mse: 1.3328 - val_loss: 65.4720 - val_accuracy: 0.0413 - val_mse: 65.4720
Epoch 351/500
1958/1958 [==============================] - 0s 148us/step - loss: 1.0329 - accuracy: 0.5041 - mse: 1.0329 - val_loss: 64.1350 - val_accuracy: 0.0550 - val_mse: 64.1350
Epoch 352/500
1958/1958 [==============================] - 0s 136us/step - loss: 1.19

Epoch 392/500
1958/1958 [==============================] - 0s 127us/step - loss: 1.5963 - accuracy: 0.4188 - mse: 1.5963 - val_loss: 62.2895 - val_accuracy: 0.0596 - val_mse: 62.2895
Epoch 393/500
1958/1958 [==============================] - 0s 123us/step - loss: 2.8766 - accuracy: 0.3636 - mse: 2.8766 - val_loss: 61.4362 - val_accuracy: 0.0505 - val_mse: 61.4362
Epoch 394/500
1958/1958 [==============================] - 0s 121us/step - loss: 2.3189 - accuracy: 0.3672 - mse: 2.3189 - val_loss: 64.0407 - val_accuracy: 0.0642 - val_mse: 64.0407
Epoch 395/500
1958/1958 [==============================] - 0s 126us/step - loss: 1.7591 - accuracy: 0.4157 - mse: 1.7591 - val_loss: 64.0430 - val_accuracy: 0.0367 - val_mse: 64.0430
Epoch 396/500
1958/1958 [==============================] - 0s 122us/step - loss: 1.2585 - accuracy: 0.4581 - mse: 1.2585 - val_loss: 60.0499 - val_accuracy: 0.0367 - val_mse: 60.0499
Epoch 397/500
1958/1958 [==============================] - 0s 122us/step - loss: 0.95

Epoch 437/500
1958/1958 [==============================] - 0s 118us/step - loss: 2.0060 - accuracy: 0.3876 - mse: 2.0060 - val_loss: 71.7814 - val_accuracy: 0.0459 - val_mse: 71.7813
Epoch 438/500
1958/1958 [==============================] - 0s 121us/step - loss: 1.5786 - accuracy: 0.4213 - mse: 1.5786 - val_loss: 64.8271 - val_accuracy: 0.0413 - val_mse: 64.8271
Epoch 439/500
1958/1958 [==============================] - 0s 119us/step - loss: 1.0692 - accuracy: 0.4939 - mse: 1.0692 - val_loss: 65.0212 - val_accuracy: 0.0505 - val_mse: 65.0212
Epoch 440/500
1958/1958 [==============================] - 0s 120us/step - loss: 1.0707 - accuracy: 0.5209 - mse: 1.0707 - val_loss: 62.0488 - val_accuracy: 0.0413 - val_mse: 62.0488
Epoch 441/500
1958/1958 [==============================] - 0s 117us/step - loss: 0.9936 - accuracy: 0.5633 - mse: 0.9936 - val_loss: 65.9381 - val_accuracy: 0.0826 - val_mse: 65.9381
Epoch 442/500
1958/1958 [==============================] - 0s 116us/step - loss: 1.53

Epoch 482/500
1958/1958 [==============================] - 0s 114us/step - loss: 1.0929 - accuracy: 0.5061 - mse: 1.0929 - val_loss: 65.4032 - val_accuracy: 0.0734 - val_mse: 65.4032
Epoch 483/500
1958/1958 [==============================] - 0s 112us/step - loss: 1.8508 - accuracy: 0.3907 - mse: 1.8508 - val_loss: 66.5953 - val_accuracy: 0.0367 - val_mse: 66.5953
Epoch 484/500
1958/1958 [==============================] - 0s 115us/step - loss: 1.8026 - accuracy: 0.4127 - mse: 1.8026 - val_loss: 63.3790 - val_accuracy: 0.0321 - val_mse: 63.3790
Epoch 485/500
1958/1958 [==============================] - 0s 113us/step - loss: 1.6747 - accuracy: 0.4249 - mse: 1.6747 - val_loss: 69.6072 - val_accuracy: 0.0183 - val_mse: 69.6072
Epoch 486/500
1958/1958 [==============================] - 0s 112us/step - loss: 1.0145 - accuracy: 0.5163 - mse: 1.0145 - val_loss: 68.3544 - val_accuracy: 0.0780 - val_mse: 68.3544
Epoch 487/500
1958/1958 [==============================] - 0s 122us/step - loss: 0.93

## XGBoost

In [12]:
for number_lags in lag_list:
    X, y = etl(number_lags)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    gbm = xgb.XGBRegressor()
    reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                              ,'max_depth': [3,4], 'n_estimators': [500,1000]}, verbose=1)
    reg_cv.fit(X_train,y_train)
    
    mlflow.set_experiment(experiment_name=ml_flow_name)
    with mlflow.start_run(run_name='XGBoost'):
        mse = mean_squared_error(y_test, reg_cv.predict(X_test))
        mlflow.log_param('lag', number_lags)
        mlflow.log_metric('val_mse', mse)

C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[12:47:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:47:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:47:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:47:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:47:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:47:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is n

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   31.0s finished


[12:47:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[12:51:45] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[12:51:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:51:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is n

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  4.6min finished


[12:56:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


## Random Forest

In [13]:
for number_lags in lag_list:
    X, y = etl(number_lags)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
    regr.fit(X_train, y_train)
    
    mlflow.set_experiment(experiment_name=ml_flow_name)
    with mlflow.start_run(run_name='Random Forest'):
        mse = mean_squared_error(y_test, regr.predict(X_test))
        mlflow.log_param('lag', number_lags)
        mlflow.log_metric('val_mse', mse)

C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\marfriz\.conda\envs\jupyterPython\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
